# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
!pip install -U datasets
!pip install -q "datasets==2.15.
!pip install scikit-learn
!pip install wandb

Defaulting to user installation because normal site-packages is not writeable
/usr/bin/sh: 1: Syntax error: Unterminated quoted string
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import os
import torch
from datetime import datetime
from datasets import load_dataset, Dataset
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification, AutoTokenizer,GPT2ForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from peft import AutoPeftModelForSequenceClassification,get_peft_model, LoraConfig,TaskType,PeftModel, PeftConfig  # Import LoRA and PEFT helper functions

# Prepare the Foundation Model

## Load a dataset

Using the same criteria applied to selecting the model, we chose the Financial PhraseBank dataset. This dataset contains sentences from financial news labeled with polar sentiments, making it suitable for our sequence classification task. We split the data into 80% for training, 10% for validation, and 10% for testing to ensure a robust evaluation of the fine-tuned model.

In [3]:
#Dataset link in hugginface
#https://huggingface.co/datasets/takala/financial_phrasebank

# Load the dataset
dataset = load_dataset("financial_phrasebank", "sentences_allagree")

# Extract sentences and labels
sentences = dataset["train"]["sentence"]
labels = dataset["train"]["label"]

# Split the data into training and test sets (80% train, 10% validation, 10% test)
#The validation set is used for evaluation during training
#The test data is used after training
train_sentences, temp_sentences, train_labels, temp_labels = train_test_split(sentences, labels, test_size=0.2, random_state=42)
val_sentences, test_sentences, val_labels, test_labels = train_test_split(temp_sentences, temp_labels, test_size=0.5, random_state=42)

# Create a dictionary for the datasets
train_dataset = {
    "sentence": train_sentences,
    "label": train_labels
}
val_dataset = {
    "sentence": val_sentences,
    "label": val_labels
}
test_dataset = {
    "sentence": test_sentences,
    "label": test_labels
}

## Load a pretrained HF model and preprocess the dataset

The chosen model for this project is GPT-2. Since we are using a free virtual environment, Google Colab, this simpler and smaller model is more suitable for applying the LoRA technique. It requires fewer resources and less time for fine-tuning, making it an ideal choice for our setup.


In [4]:
# Load the GPT-2
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Add the padding token to the tokenizer (GPT-2 doesn't have a padding token by default)
tokenizer.pad_token = tokenizer.eos_token  # Set eos_token as pad token for GPT-2

# Load the pre-trained model for sequence classification
pretrained_model = GPT2ForSequenceClassification.from_pretrained(model_name, num_labels=len(set(labels)))

# Crucially, update the model config's pad_token_id
pretrained_model.config.pad_token_id = tokenizer.pad_token_id  # Ensure model has the same pad_token_id as tokenizer

/home/student/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# Step 3: Define a function to tokenize the data
def tokenize_data(batch):
    return tokenizer(batch["sentence"], padding="max_length", truncation=True, max_length=128)


train_dataset = Dataset.from_dict(train_dataset).map(tokenize_data, batched=True)
val_dataset = Dataset.from_dict(val_dataset).map(tokenize_data, batched=True)
test_dataset = Dataset.from_dict(test_dataset).map(tokenize_data, batched=True)

# Set the format for PyTorch
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

Map:   0%|          | 0/1811 [00:00<?, ? examples/s]

Map:   0%|          | 0/226 [00:00<?, ? examples/s]

Map:   0%|          | 0/227 [00:00<?, ? examples/s]

## Evaluate the pretrained model

For the evaluation, we use accuracy and F1 score as the primary metrics.

In [6]:

# Evaluation function
def evaluate_model(model, dataloader):
    model.eval()
    predictions, true_labels = [], []
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["label"].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            preds = torch.argmax(outputs.logits, dim=1)
            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions, average="weighted")
    return {"accuracy": accuracy, "f1": f1}

# Load data into DataLoader
from torch.utils.data import DataLoader
test_loader = DataLoader(test_dataset, batch_size=16)

# Evaluate pre-trained model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pretrained_model.to(device)
metrics_pre = evaluate_model(pretrained_model, test_loader)
print("Pre-trained Model Metrics:", metrics_pre)


Pre-trained Model Metrics: {'accuracy': 0.14537444933920704, 'f1': 0.11116869655351126}


# Perform Lightweight Fine-Tuning

## Create a PEFT model

In [7]:
# Configure LoRA for fine-tuning
lora_config = LoraConfig(
    r=8,  # Rank for LoRA matrices
    lora_alpha=32,  # Scaling factor
    target_modules=["attn.c_attn", "attn.c_proj"],  # Layers where LoRA will be applied
    task_type=TaskType.SEQ_CLS,  # Task type: Sequence Classification
)

# Apply LoRA to the pre-trained model
model_with_lora = get_peft_model(pretrained_model, lora_config)

/opt/conda/lib/python3.10/site-packages/peft/tuners/lora.py:475: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


## Train the PEFT model

In [8]:
model_with_lora.print_trainable_parameters()

trainable params: 446,976 || all params: 124,886,784 || trainable%: 0.3579049645477299


In [9]:
# Step 3: Define a function to tokenize the data
def tokenize_data(batch):
    return tokenizer(batch["sentence"], padding="max_length", truncation=True, max_length=128)

# Load data into DataLoader
test_loader = DataLoader(test_dataset, batch_size=16)

output_dir = './results'

# Train the model (fine-tuning with LoRA)
training_args = TrainingArguments(
    output_dir= output_dir,          # Output directory for checkpoints
    evaluation_strategy="epoch",     # Evaluate every epoch
    learning_rate=2e-5,              # Learning rate
    per_device_train_batch_size=8,   # Batch size for training
    per_device_eval_batch_size=8,    # Batch size for evaluation
    num_train_epochs=5,              # Number of epochs
    weight_decay=0.01,               # Weight decay
    report_to="wandb",                 # Ensure training logs are reported to W&B
    run_name="Parameter-efficient fine-tuning"   # Set the name for the W&B run
)

trainer = Trainer(
    model=model_with_lora,           # LoRA model to train
    args=training_args,             # Training arguments
    train_dataset=train_dataset,    # Training dataset
    eval_dataset=val_dataset,       # Evaluation dataset
)

# Train the model
trainer.train()

model_with_lora.save_pretrained("gpt-lora")

# Evaluate the fine-tuned model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_with_lora.to(device)
metrics_pre = evaluate_model(model_with_lora, test_loader)
print("Fine-tuned LoRA Model Metrics:", metrics_pre)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: alsotti (alsotti-udacity). Use `wandb login --relogin` to force relogin
/home/student/.local/lib/python3.10/site-packages/pydantic/main.py:314: UserWarning: Pydantic serializer warnings:
  Expected `list[str]` but got `tuple` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


Epoch,Training Loss,Validation Loss
1,No log,0.858079
2,No log,0.686007
3,1.092700,0.623519
4,1.092700,0.588727
5,0.626800,0.583019


Checkpoint destination directory ./results/checkpoint-500 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-1000 already exists and is non-empty.Saving will proceed but saved results may be invalid.


Fine-tuned LoRA Model Metrics: {'accuracy': 0.7533039647577092, 'f1': 0.7128138995456812}


In [10]:
loaded_model = AutoPeftModelForSequenceClassification.from_pretrained("gpt-lora",num_labels=3)
loaded_model.config.pad_token_id = tokenizer.pad_token_id  # Ensure model has the same pad_token_id as tokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loaded_model.to(device)

metrics_loaded = evaluate_model(loaded_model, test_loader)
print("Reloaded LoRA Model Metrics:", metrics_loaded)

/home/student/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Reloaded LoRA Model Metrics: {'accuracy': 0.7533039647577092, 'f1': 0.7128138995456812}
